In [11]:
import os
import json
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.datasets.fashion_mnist import load_data

from surprise_adequacy import SurpriseAdequacyConfig, DSA, LSA

In [2]:
(x_train, _), (x_test, _) = load_data()

In [3]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.0
x_test = x_test / 255.0

In [7]:
model = load_model('./models/model_01/model.h5')

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1605760   
                                                        

In [10]:
with open('./models/model_01/training_history.json', 'r') as f:
    history = json.load(f)
    
plt.plot(history['loss'])
# plt.plot(history['val_loss'])
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './models/model_01/training_history.json'

In [15]:
config = SurpriseAdequacyConfig(saved_path='./tmp', is_classification=True, num_classes=10, 
                                layer_names=["dense_1"], ds_name='fashion_mnist')

In [16]:
# calculating LSA

sa = LSA(model=model, train_data=x_train, config=config)
sa.prep(use_cache=False)

surprises, predictions = sa.calc(target_data=x_test, use_cache=False, ds_type='test')

surprises = np.argsort(surprises)[::-1]

np.save('./tmp/lsa.npy', surprises)

Calculating the ats for train dataset
469/469 [==============================] - 17s 37ms/step


kde:   0%|          | 0/10 [00:00<?, ?it/s]

Layer: dense_1
[train] Saved the ats and predictions to ./tmp/fashion_mnist_train_dense_1_ats.npy and ./tmp/fashion_mnist_train_pred.npy


kde: 100%|██████████| 10/10 [00:00<00:00, 512.39it/s]

Ignoring the activations of 0 traces as their variance is not high enough.
Calculating the ats for test dataset
 1/79 [..............................] - ETA: 0s

79/79 [==============================] - 3s 38ms/step
Layer: dense_1
[test] Saved the ats and predictions to ./tmp/fashion_mnist_test_dense_1_ats.npy and ./tmp/fashion_mnist_test_pred.npy
[test] Calculating LSA


In [17]:
# calculating DSA

sa = DSA(model=model, train_data=x_train, config=config)
sa.prep(use_cache=False)

surprises, predictions = sa.calc(target_data=x_test, use_cache=False, ds_type='test')

surprises = np.argsort(surprises)[::-1]

np.save('./tmp/dsa.npy', surprises)

Calculating the ats for train dataset
469/469 [==============================] - 19s 40ms/step
Layer: dense_1
[train] Saved the ats and predictions to ./tmp/fashion_mnist_train_dense_1_ats.npy and ./tmp/fashion_mnist_train_pred.npy
Calculating the ats for test dataset
79/79 [==============================] - 3s 39ms/step
Layer: dense_1
[test] Saved the ats and predictions to ./tmp/fashion_mnist_test_dense_1_ats.npy and ./tmp/fashion_mnist_test_pred.npy
[test] Calculating DSA
[<class 'surprise_adequacy.DSA'>] Using 60000 train samples
